In [3]:
import inspect

class Validator:
    
    _min_val = 0
    _max_val = 0 
    
    def __init__(self, min_val=0, max_val=0):
        _min_val = min_val
        _max_val = max_val
        
    @classmethod
    def validateMax(cls, value):
        if isinstance(value, int):
            return value if value <= cls._max_val else cls._max_val
        return cls._max_val
    
    @classmethod
    def validateMin(cls, value):
        if isinstance(value, int):
            return value if value >= cls._min_value else cls._min_val
        return cls._min_val
        
    
    
class IntValidator(Validator):
    
    _min_val = -2**31
    _max_val = 2**31 - 1
    
    def __init__(self, min_val=-2**31, max_val=2**31 - 1):
        super().__init__(min_val, max_val)
    
    
class CharValidator(Validator):
    
    _min_val = 1
    _max_val = 8000 
    
    def __init__(self, min_val=1, max_val=8000):
        super().__init__(min_val, max_val)
    
        
class FloatValidator(Validator):
    
    _min_val = -1.79e+308  
    _max_val = 1.79e+308
    
    def __init__(self, min_val=-1.79e+308, max_val=1.79e+308):
        super()._init_(min_val, max_val)
        
        
class Field:
    
    def __init__(self, name='', cls=None, required=True, default=None):
        self.name = '_' + name
        self.cls = cls
        self.required = required
        self.default = self.validate(default)
        
    def __get__(self, instance, owner=None):
        if owner is None:
            return AttributeError('This field must be defined in a class!')
        return getattr(instance, self.name, self.default)
    
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
    
    def __delete__(self, instance):
        if instance is None:
            raise AttributeError()
        del instance.__dict__[self.name]
        
    def validate(self, value):
        if value is None and not self.required:
            return None
        return self.cls(value)
    
    def cls_validate(self, value):
        if not isinstance(value, self.cls):
            raise AttributeError('The value have to be {0}'.format(self.cls))
        return 

        
class IntField(Field):
    
    def __init__(self, name='', min_value=None, max_value=None, **kwarg):
        self.min_value = IntValidator.validateMin(min_value)
        self.max_value = IntValidator.validateMax(max_value)
        super().__init__(name, int, **kwarg)
        
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
        
    def __set__(self, instance, value):
        super().cls_validate(value)
       
        if self.min_value > value or self.max_value < value:
            raise AttributeError(
                'Value have to lay between {0} and {1}'.format(
                    self.min_value,
                    self.max_value
                )
            )
        super().__set__(instance, value)

    def __delete__(self, instance):
        super().__delete__(instance)
        
        
class CharField(Field):
    
    def __init__(self, name='', max_length=None, min_length=None, **kwarg):
        self.min_length = CharValidator.validateMin(min_length)
        self.max_length = CharValidator.validateMax(max_length)
        super().__init__(name, str, **kwarg)
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        super().cls_validate(value)
        
        if self.min_length > value or self.max_length < value:
            raise AttributeError(
                'Value have to lay between {0} and {1}'.format(
                    self.min_length,
                    self.max_lenght
                )
            )
        super().__set__(instance, value)
        
    def __delete__(self, instance):
        super().__delete__(instance)
        
        
class BoolField(Field):
    
    def __init__(self, name='', **kwarg):
        super().__init__(name, bool, **kwarg)
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        super().cls_validate(value)
        super().__set__(instance, value)
        
    def __delete__(self, instance):
        super().__delete__(instance)
        
        
class FloatField(Field):
    
    def __init__(self, name='', min_val=None, max_val=None, **kwarg):
        self.min_val = CharValidator.validateMin(min_val)
        self.max_val = CharValidator.validateMax(max_val)
        super().__init__(name, float, **kwarg)
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        super().cls_validate(value)
        super().__set__(instance, value)
        
    def __delete__(self, instance):
        super().__delete__(instance)
        
        
class AutoField(IntField):
    # создаем словарь из названий класов и количества экземпляров автофилда
    # пропишем автоинкрементацию в классе модели
    _count_dict = {}
    
    def __init__(self, name='', **kwarg):
        kwarg['required'] = True
        
        if kwarg.get('default', None) is not None:
            raise AttributeError(
                'AutoField does not contain default value!'
            )
        super().__init__(name, min_val=0, max_val=2**31-1, **kwarg)
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        super().__set__(instance, value)
    
    def __delete__(self, instance):
        super().__delete__(instance)
        
    def _set_number(self, field_name=None):
        if field_name is None:
            raise AttributeError(
                '{0} model has no AutoField'.format(field_name)
            )

In [4]:
class MetaModel(type):
    
    def __new__(mcs, name, bases, namespace):
        super_new = super().__new__
        
        # To choose only bases class constructed by MetaModel
        parents = tuple([b for b in bases if isinstance(b, MetaModel)])
        if not parents:
            return super_new(mcs, name, bases, namespace)
        
        new_namespace = dict()
        module = namespace.pop('__module__', None)
        if module is not None:
            new_namespace['__module__'] = module
            
        classcell = namespace.pop('__classcell__', None)
        if classcell is not None:
            new_namespace['__classcell__'] = classcell

        new_class = super_new(mcs, name, parents, new_namespace)
        
        fields = {name: value for name, value in namespace.items() if isinstance(value, Field)}
        methods = {name: value for name, value in namespace.items() if inspect.isfunction(value)}
        fields_name = set(fields.keys())
        
        meta = namespace.pop('Meta', None)
        if meta is not None:
            if not hasattr(meta, 'table_name'):
                setattr(meta, 'table_name', name)
            # There shoulde change conditions
            if not hasattr(meta, 'fields'):
                setattr(meta, 'fields', fields_name)
        else:
            meta = type('Meta', tuple(), {'table_name' : name, 'fields' : fields_name})
        
        # To inherite the fields by MRO
        inherited_namespace = dict()
        for base_cls in new_class.mro():
            for field_name, value in base_cls.__dict__.items():                 
                if isinstance(value, Field) or inspect.isfunction(value):
                    inherited_namespace.update({field_name: value})
        
        meta.fields.update(set(inherited_namespace.keys()))
        fields.update(inherited_namespace)
        fields.update(methods)
        
        # To add meta-data
        setattr(new_class, 'Meta', meta)
        
        # To set fields from base classes
        for attr_name, attr_value in fields.items():
            setattr(new_class, attr_name, attr_value)

        return new_class
            

In [12]:
class Model(metaclass=MetaModel):
    pass

class User(Model):
    Id = IntField('Id', required=True, default=2)
    Name = CharField('Name', required=True)
    sex = BoolField('sex', required=True, default=True)
    Money  = FloatField('Money', required=False, default=0.0)
    
    def update_money(self, value):
        self.Money = value
    
a = User()
a.Id=3

class Manager(User):
    Post = IntField('Post', required=True, default = 0)
    
    
class A(Model):
    pass

mappingproxy({'table_name': 'A',
              'fields': set(),
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Meta' objects>,
              '__weakref__': <attribute '__weakref__' of 'Meta' objects>,
              '__doc__': None})

In [135]:
class A():
    a = 0
    b = ""
    c = True
    
    def __init__(self, a, b, c):
        print("Called fron {}".format(self))
        self.a = a
        self.b = b
        self.c = c
        
    def update_a(self, a):
        print("Called fron {}".format(self))
        self.a = a
    
    def update_b(self, b):
        print("Called fron {}".format(self))
        self.b = b
        
    def update_c(self, c):
        print("Called fron {}".format(self))
        self.c = c
        
class MetaClass(type):
    def __new__(mcs, name, bases, attr):
        meta = attr.get('Meta')
        print(attr)
        if meta is not None:
            print(meta)
            print(hasattr(attr['Meta'], 'table_name'))
        return super().__new__(mcs, name, bases, attr)
        
class B(A):
    
    class Meta:
        table_name = 'B'
        fields = ('d')
        
    a = 'a'
    
    def __init__(self, a, b, c, d):
        print("Called fron {}".format(self))
        self.a = d
        super().__init__(a, b, c)
        
    def lel(self):
        return 1
        
        

In [136]:
b = B(1,2,3,4)
inspect.isfunction(b.__class__.__dict__['lel'])

Called fron <__main__.B object at 0x7fdf9a1c3208>
Called fron <__main__.B object at 0x7fdf9a1c3208>


True

In [82]:
B(1,2,3,4).__class__.__dict__['Meta'].__dict__['table_name']

Called fron <__main__.B object at 0x7ff93c273588>
Called fron <__main__.B object at 0x7ff93c273588>


'B'

In [88]:
a= {'a': 1, 'b' : 2}
set(a.keys())

{'a', 'b'}

In [86]:
a = B(1,2,3,4)
isinstance(B.mro()[0], MetaClass)

Called fron <__main__.B object at 0x7ff93c2210b8>
Called fron <__main__.B object at 0x7ff93c2210b8>


True

In [136]:
-2.23e-308
        

-2.23e-308

In [15]:
import sys
print(sys.version)

2.7.15 |Anaconda, Inc.| (default, Dec 14 2018, 19:04:19) 
[GCC 7.3.0]


In [14]:
class A:
    pass
        
a = [A(), A(), A()]
dir(a.__class__)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']